##Setup from Intro to ML Course

In [ ]:
import pandas as pd;
import numpy as np;
import warnings; warnings.simplefilter('ignore');
from sklearn.model_selection import train_test_split;
X_full=pd.read_csv("/content/drive/My Drive/Dataset/train.csv",index_col='Id');
X_test=pd.read_csv("/content/drive/My Drive/Dataset/test.csv" ,index_col='Id');

In [ ]:
#Seperating dependent variable from dataset
X_full.dropna(axis=0,subset=['SalePrice'],inplace=True);
y=X_full['SalePrice'];
X_full.drop(axis=1,columns=['SalePrice'],inplace=True);

In [ ]:
#Creating function to get mean absolute error for dataset
from sklearn.ensemble import RandomForestRegressor;
from sklearn.metrics  import mean_absolute_error;
def score_dataset(x_train,x_valid,y_train,y_valid):
  model=RandomForestRegressor(n_estimators=100,random_state=0);
  model.fit(x_train,y_train);
  y_pred=model.predict(x_valid);
  return mean_absolute_error(y_pred,y_valid);

In [ ]:
#Creating function to get cross fold validation scores
from sklearn.model_selection import cross_val_score;
from sklearn.linear_model import LinearRegression;
def k_fold_validation(X,y):
  my_model=RandomForestRegressor(n_estimators=100);
  scores=-1*cross_val_score(estimator=my_model,X=X,y=y,cv=10,scoring='neg_mean_absolute_error');
  return (scores.mean(),scores.std());

##Handling Missing Value

In [ ]:
# for sake of simplicity removing categorical column
train=X_full.select_dtypes(exclude=['object']);
test =X_test.select_dtypes(exclude=['object']);
x_train,x_valid,y_train,y_valid=train_test_split(train,y,test_size=0.2,random_state=0);

###Dropping Column with missing value

In [ ]:
missing_col=[col for col in x_train.columns if x_train[col].isnull().any()]
reduced_train=x_train.drop(axis=1,columns=missing_col);
reduced_valid=x_valid.drop(axis=1,columns=missing_col);
print(score_dataset(reduced_train,reduced_valid,y_train,y_valid));

17837.82570776256


###Imputation

In [ ]:
from sklearn.impute import SimpleImputer;
imputer=SimpleImputer();
imputed_train=x_train;
imputed_train.loc[:,missing_col]=imputer.fit_transform(imputed_train.loc[:,missing_col]);
imputed_valid=x_valid;
imputed_valid.loc[:,missing_col]=imputer.transform(imputed_valid.loc[:,missing_col]);
print(score_dataset(imputed_train,imputed_valid,y_train,y_valid));

18062.894611872147


##Handling Categorical Variables

In [ ]:
#for sake of simplicity, removing column with missing value
missing_col=[col for col in X_full if X_full[col].isnull().any()];
train=X_full.drop(axis=1,columns=missing_col);
test =X_test.drop(axis=1,columns=missing_col);
x_train,x_valid,y_train,y_valid=train_test_split(train,y,test_size=0.2,random_state=0);

###Label Encoding

In [ ]:
#We can't use label encoding on every column, may be valid data have some new variable that is not present in train data for example, next 2 lines
#print(x_train.Condition2.unique());
#print(x_valid.Condition2.unique());
object_cols=[col for col in x_train if x_train[col].dtype=='object'];
good_label_cols=[col for col in x_train if (set(x_valid[col])).issubset(set(x_train[col]))];
bad_label_cols =list(set(object_cols)-set(good_label_cols));

In [ ]:
from sklearn.preprocessing import LabelEncoder;
label_train=x_train.drop(axis=1,columns=bad_label_cols);
label_valid=x_valid.drop(axis=1,columns=bad_label_cols);
lb_encoder=LabelEncoder();
for col in good_label_cols:
  label_train[col]=lb_encoder.fit_transform(label_train[col]);
  label_valid[col]=lb_encoder.transform(label_valid[col]);
print(score_dataset(label_train,label_valid,y_train,y_valid));
print(k_fold_validation(pd.concat([label_train,label_valid]),pd.concat([y_train,y_valid])));

17096.280194063926
(17453.72759784736, 1824.794892428798)


In [ ]:
from sklearn.preprocessing import OneHotEncoder;
ht_encoder=OneHotEncoder(handle_unknown='ignore',sparse=False);
hot_x_train=x_train.drop(axis=1,columns=bad_label_cols);
hot_x_valid=x_valid.drop(axis=1,columns=bad_label_cols);
hot_train_cols=pd.DataFrame(ht_encoder.fit_transform(hot_x_train[good_label_cols]));
hot_valid_cols=pd.DataFrame(ht_encoder.transform(hot_x_valid[good_label_cols]));
hot_train_cols.index=hot_x_train.index;
hot_valid_cols.index=hot_x_valid.index;
num_x_train=x_train.drop(axis=1,columns=object_cols);
num_x_valid=x_valid.drop(axis=1,columns=object_cols);
hot_x_train=pd.concat([hot_train_cols,num_x_train],axis=1);
hot_x_valid=pd.concat([hot_valid_cols,num_x_valid],axis=1);
print(score_dataset(hot_x_train,hot_x_valid,y_train,y_valid))
print(k_fold_validation(pd.concat([hot_x_train,hot_x_valid]),pd.concat([y_train,y_valid])));

7658923.376712329
(785917421.3218677, 1611505556.8068984)


###One Hot Encoding

In [ ]:
#Investing Cardinality
object_nunique=list(map(lambda col : x_train[col].nunique(),object_cols))
d=dict(zip(object_cols,object_nunique));
low_cardinal_cols=[col for col in object_cols if x_train[col].nunique()<10];
high_cardinal_cols=list(set(object_cols)-set(low_cardinal_cols));

In [ ]:
from sklearn.preprocessing import OneHotEncoder;
ht_encoder=OneHotEncoder(handle_unknown='ignore',sparse=False);
hot_x_train=x_train.drop(axis=1,columns=high_cardinal_cols);
hot_x_valid=x_valid.drop(axis=1,columns=high_cardinal_cols);
hot_train_cols=pd.DataFrame(ht_encoder.fit_transform(hot_x_train[low_cardinal_cols]));
hot_valid_cols=pd.DataFrame(ht_encoder.transform(hot_x_valid[low_cardinal_cols]));
hot_train_cols.index=hot_x_train.index;
hot_valid_cols.index=hot_x_valid.index;
num_x_train=x_train.drop(axis=1,columns=object_cols);
num_x_valid=x_valid.drop(axis=1,columns=object_cols);
hot_x_train=pd.concat([hot_train_cols,num_x_train],axis=1);
hot_x_valid=pd.concat([hot_valid_cols,num_x_valid],axis=1);
print(score_dataset(hot_x_train,hot_x_valid,y_train,y_valid))

17537.55444063927


##Pipelines

In [ ]:
#Keeping seperate list of numerical_col, categorical_col
num_cols=[col for col in X_full.columns if X_full[col].dtype in ['int64','float64']];
cat_cols=[col for col in X_full.columns if X_full[col].nunique()<10 and X_full[col].dtype=='object'];
my_cols=num_cols+cat_cols;
train=X_full[my_cols].copy();
test =X_test[my_cols].copy();
x_train,x_valid,y_train,y_valid=train_test_split(train,y,test_size=0.2,random_state=0);

In [ ]:
from sklearn.compose import ColumnTransformer;
from sklearn.impute import SimpleImputer;
from sklearn.preprocessing import OneHotEncoder;
from sklearn.pipeline import Pipeline;
from sklearn.ensemble import RandomForestRegressor;
from sklearn.metrics import mean_absolute_error;
num_transformer=SimpleImputer(strategy='mean');
cat_transformer=Pipeline(steps=[
                                ('imputer',SimpleImputer(strategy='most_frequent')),
                                ('encoder',OneHotEncoder(sparse=False,handle_unknown='ignore'))]);
precomputer=ColumnTransformer(transformers=[
                                            ('num',num_transformer,num_cols),
                                            ('cat',cat_transformer,cat_cols)]);
complete_model=Pipeline(steps=[
                               ('preprocessor',precomputer),
                               ('model',RandomForestRegressor(n_estimators=100,random_state=0))]);
complete_model.fit(x_train,y_train);
y_pred=complete_model.predict(x_valid);
print(mean_absolute_error(y_pred,y_valid));

17648.417157534244
